# `azureai_sentiment`

## Overview
The `azureai_sentiment` Python function uses the [Azure AI Language](https://azure.microsoft.com/en-us/pricing/details/cognitive-services/language-service/) service to analyze sentiment and opinions in the provided documents. It returns an array containing the sentiment analysis results with sentiment polarity at the document, sentence, and aspect levels. It also provides the opinions used at the aspect level.  

See this [demo workbook](https://whistlernetworks.sharepoint.com/:x:/s/Boardflare/EZOwMhafmQhNktK7qkD3-mABf-dJtDrPGRXgB3pEshzhAA?e=4LPE3z) for example usage.  Edit source code in [Jupyterlite](https://addins.boardflare.com/functions/prod/jupyterlite/lab/index.html?path=text/sentiment-analysis/azure_sentiment.ipynb).

## Compatibility
BOARDFLARE.RUNPY ✅  
Python in Excel ❌

## Usage

Arguments:

| Argument      | Positional | Type           | Description                                                                 |
|---------------|------------|----------------|-----------------------------------------------------------------------------|
| `documents`   | arg1       | string or list | The text documents to analyze. Can be a single string or a pandas DataFrame.|
| `azureai_key` | arg2       | string         | The Azure AI API key. e.g. 7920c673894d49d292de11a80d16572                  |
| `azureai_url` | arg3       | string         | The Azure AI API endpoint (e.g. https://boardflare.openai.azure.com/)       |

Returns a list of lists containing the sentiment analysis results. Each inner list contains:

| Return Value     | Type  | Description                                                                                  |
|------------------|-------|----------------------------------------------------------------------------------------------|
| DocIndex         | int   | Document index (1-based).                                                                    |
| DSent            | string| Document sentiment.                                                                          |
| DPos             | float | Document confidence score (positive).                                                        |
| DNeu             | float | Document confidence score (neutral).                                                         |
| DNeg             | float | Document confidence score (negative).                                                        |
| Sentence         | string| Sentence text.                                                                               |
| SSent            | string| Sentence sentiment.                                                                          |
| SPos             | float | Sentence confidence score (positive).                                                        |
| SNeu             | float | Sentence confidence score (neutral).                                                         |
| SNeg             | float | Sentence confidence score (negative).                                                       |
| Aspect           | string| Target text.                                                                                 |
| ASent            | string| Target sentiment.                                                                            |
| APos             | float | Target confidence score (positive).                                                          |
| ANeg             | float | Target confidence score (negative).                                                          |
| Opinion          | string| Assessment text.                                                                             |
| OSent            | string| Assessment sentiment.                                                                        |
| OPos             | float | Assessment confidence score (positive).                                                      |
| ONeg             | float | Assessment confidence score (negative).                                                      |

### BOARDFLARE.RUNPY

```excel
=BOARDFLARE.RUNPY("text/sentiment/azureai_sentiment.ipynb", documents, azureai_key, azureai_url)
```
For example, to analyze the sentiment of a list of documents using your Azure AI key and endpoint, you would use the following formula:

```excel
=BOARDFLARE.RUNPY("text/sentiment/azureai_sentiment.ipynb", A1:A10, "your_azureai_key", "your_azureai_url")
```

### LAMBDA

**AZUREAI.SENTIMENT**

User-friendly named LAMBDA in which you could hard code the azureai_key and azureai_url to further simplify its use.

```excel
=LAMBDA(documents, azureai_key, azureai_url,
    BOARDFLARE.RUNPY("text/sentiment/azureai_sentiment.ipynb", documents, azureai_key, azureai_url)
)
```

For example, to analyze the sentiment of a list of documents using your Azure AI key and endpoint, you would use the following formula:
```excel
=AZUREAI.SENTIMENT(A1, B1:B10, 0.8)
```

## How it works

### Sentiment Analysis
Sentiment Analysis applies sentiment labels to text, which are returned at a sentence and document level, with a confidence score for each.

The labels are positive, negative, and neutral. At the document level, the mixed sentiment label also can be returned. The sentiment of the document is determined below:

| Sentence sentiment | Returned document label |
|--------------------|-------------------------|
| At least one `positive` sentence is in the document. The rest of the sentences are `neutral`. | `positive` |
| At least one `negative` sentence is in the document. The rest of the sentences are `neutral`. | `negative` |
| At least one `negative` sentence and at least one `positive` sentence are in the document. | `mixed` |
| All sentences in the document are `neutral`. | `neutral` |

Confidence scores range from 1 to 0. Scores closer to 1 indicate a higher confidence in the label's classification, while lower scores indicate lower confidence. For each document or each sentence, the predicted scores associated with the labels (positive, negative, and neutral) add up to 1. For more information, see the Responsible AI transparency note.

### Opinion Mining
Opinion Mining is a feature of Sentiment Analysis. Also known as Aspect-based Sentiment Analysis in Natural Language Processing (NLP), this feature provides more granular information about the opinions related to attributes of products or services in text. The API surfaces opinions as a target (noun or verb) and an assessment (adjective).

For example, if a customer leaves feedback about a hotel such as "The room was great, but the staff was unfriendly.", Opinion Mining will locate targets (aspects) in the text, and their associated assessments (opinions) and sentiments. Sentiment Analysis might only report a negative sentiment.

![](https://learn.microsoft.com/en-us/azure/ai-services/language-service/sentiment-opinion-mining/media/opinion-mining.png)


In [1]:
%pip install micropip
import micropip
import pandas as pd
import json

In [2]:
# arg1: Documents to analyze (column array)
arg1 = pd.DataFrame([
    "The food and service were unacceptable. The concierge was nice, however.",
    "The rooms were clean and well maintained.",
    "The location is perfect but the staff was rude.",
    "The breakfast was delicious but the coffee was too strong.",
    "The Wi-Fi was fast and reliable, but the room was too small.",
    "The gym facilities were excellent, but the pool was dirty.",
    "The check-in process was smooth, but the checkout was slow.",
    "The bed was comfortable, but the pillows were too soft.",
    "The air conditioning worked well, but it was noisy.",
    "The parking was convenient, but it was expensive."
])

# arg1: Documents to analyze (single string)
# arg1 = "The food and service were unacceptable. The concierge was nice, however."

# output arg1 as column vector
json.dumps([[x] for x in arg1[0].values.tolist()])

'[["The food and service were unacceptable. The concierge was nice, however."], ["The rooms were clean and well maintained."], ["The location is perfect but the staff was rude."], ["The breakfast was delicious but the coffee was too strong."], ["The Wi-Fi was fast and reliable, but the room was too small."], ["The gym facilities were excellent, but the pool was dirty."], ["The check-in process was smooth, but the checkout was slow."], ["The bed was comfortable, but the pillows were too soft."], ["The air conditioning worked well, but it was noisy."], ["The parking was convenient, but it was expensive."]]'

In [3]:
# arg2: Azure language api key
arg2 = '7920c673894d49d292de11a80d165725'
arg2

'7920c673894d49d292de11a80d165725'

In [4]:
# arg3: Azure language api endpoint
arg3 = 'https://boardflare.openai.azure.com/'
arg3

'https://boardflare.openai.azure.com/'

In [5]:
await micropip.install(["azure-ai-textanalytics"])
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Example method for detecting sentiment and opinions in text 
def azureai_sentiment(documents, language_key, language_endpoint):
    """
    Analyzes sentiment and opinions in the provided documents using Azure Text Analytics.

    Parameters:
    documents (str or DataFrame): The text documents to analyze. Can be a single string or a pandas DataFrame.
    language_key (str): The Azure Text Analytics API key.
    language_endpoint (str): The Azure Text Analytics API endpoint.

    Returns:
    list: A list of lists containing the sentiment analysis results. Each inner list contains:
        - Document index (1-based)
        - Document sentiment
        - Document confidence scores (positive, neutral, negative)
        - Sentence text
        - Sentence sentiment
        - Sentence confidence scores (positive, neutral, negative)
        - Target text
        - Target sentiment
        - Target confidence scores (positive, negative)
        - Assessment text
        - Assessment sentiment
        - Assessment confidence scores (positive, negative)
    """
    # Check if documents is a string, list, or DataFrame
    if isinstance(documents, str):
        documents = [documents]
    elif isinstance(documents, pd.DataFrame):
        documents = documents.values.flatten().tolist()  # Convert df to list

    # Authenticate the client using your key and endpoint 
    ta_credential = AzureKeyCredential(language_key)
    client = TextAnalyticsClient(
            endpoint=language_endpoint, 
            credential=ta_credential)

    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    data = []

    for idx, document in enumerate(doc_result):
        for sentence in document.sentences:
            for mined_opinion in sentence.mined_opinions:
                target = mined_opinion.target
                for assessment in mined_opinion.assessments:
                    data.append([
                        idx + 1,  # 1-based index
                        document.sentiment,
                        document.confidence_scores.positive,
                        document.confidence_scores.neutral,
                        document.confidence_scores.negative,
                        sentence.text,
                        sentence.sentiment,
                        sentence.confidence_scores.positive,
                        sentence.confidence_scores.neutral,
                        sentence.confidence_scores.negative,
                        target.text,
                        target.sentiment,
                        target.confidence_scores.positive,
                        target.confidence_scores.negative,
                        assessment.text,
                        assessment.sentiment,
                        assessment.confidence_scores.positive,
                        assessment.confidence_scores.negative
                    ])
    return data

azureai_sentiment(arg1, arg2, arg3)

[[1,
  'mixed',
  0.3,
  0.17,
  0.52,
  'The food and service were unacceptable. ',
  'negative',
  0.0,
  0.0,
  1.0,
  'food',
  'negative',
  0.01,
  0.99,
  'unacceptable',
  'negative',
  0.01,
  0.99],
 [1,
  'mixed',
  0.3,
  0.17,
  0.52,
  'The food and service were unacceptable. ',
  'negative',
  0.0,
  0.0,
  1.0,
  'service',
  'negative',
  0.01,
  0.99,
  'unacceptable',
  'negative',
  0.01,
  0.99],
 [1,
  'mixed',
  0.3,
  0.17,
  0.52,
  'The concierge was nice, however.',
  'positive',
  0.61,
  0.35,
  0.05,
  'concierge',
  'positive',
  1.0,
  0.0,
  'nice',
  'positive',
  1.0,
  0.0],
 [2,
  'positive',
  0.86,
  0.14,
  0.0,
  'The rooms were clean and well maintained.',
  'positive',
  0.86,
  0.14,
  0.0,
  'rooms',
  'positive',
  1.0,
  0.0,
  'clean',
  'positive',
  1.0,
  0.0],
 [2,
  'positive',
  0.86,
  0.14,
  0.0,
  'The rooms were clean and well maintained.',
  'positive',
  0.86,
  0.14,
  0.0,
  'rooms',
  'positive',
  1.0,
  0.0,
  'well main

In [6]:
# Outputs docstring
azureai_sentiment.__doc__

'\n    Analyzes sentiment and opinions in the provided documents using Azure Text Analytics.\n\n    Parameters:\n    documents (str or DataFrame): The text documents to analyze. Can be a single string or a pandas DataFrame.\n    language_key (str): The Azure Text Analytics API key.\n    language_endpoint (str): The Azure Text Analytics API endpoint.\n\n    Returns:\n    list: A list of lists containing the sentiment analysis results. Each inner list contains:\n        - Document index (1-based)\n        - Document sentiment\n        - Document confidence scores (positive, neutral, negative)\n        - Sentence text\n        - Sentence sentiment\n        - Sentence confidence scores (positive, neutral, negative)\n        - Target text\n        - Target sentiment\n        - Target confidence scores (positive, negative)\n        - Assessment text\n        - Assessment sentiment\n        - Assessment confidence scores (positive, negative)\n    '

In [7]:
# Column headers to use in demo workbook.
headers = ["Documents(arg1)", "AzureKey(arg2)", "AzureURL(arg3)", "DocIndex", "DSent", "DPos", "DNeu", "DNeg", "Sentence", "SSent", "SPos", "SNeu", "SNeg", "Aspect", "ASent", "APos", "ANeg", "Opinion", "OSent", "OPos", "ONeg"]
json.dumps(headers)

'["Documents(arg1)", "AzureKey(arg2)", "AzureURL(arg3)", "DocIndex", "DSent", "DPos", "DNeu", "DNeg", "Sentence", "SSent", "SPos", "SNeu", "SNeg", "Aspect", "ASent", "APos", "ANeg", "Opinion", "OSent", "OPos", "ONeg"]'